<br>
<a href="https://www.nvidia.cn/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>使用大语言模型（LLM）构建 AI 智能体</h1>
<h2><b>Notebook 2:</b> 组织思考和输出</h2>
<br>

**欢迎回到课程中！这是课程的第二个主要部分，希望您准备好开始了！**

之前的 notebook 是让您了解基本的智能体聊天循环，并且稍微涉及了一下智能体分解。这部分将深入探讨 LLM 模型的能力，以弄清楚到底能对它抱有什么期待。具体来说，我们想知道它真正能推理什么，能够多好地考虑输入，以及这对我们与外部（甚至内部）环境的交互能力意味着什么。毕竟是希望拥有一些能够可靠运行的 LLM 组件，而不仅仅做语义推理器。

### **学习目标：**
**这个 notebook 将：**

- 调查 LLM 接口，并考虑它似乎能做些什么，以及我们如何尝试使用它。
- 对于其表现不佳的情况，考虑可能的原因，看看能否找到解决方法。
- 最重要的是，了解如何“保证”模型能在给定接口中输出，以及在语义推理的背景下这实际上意味着什么。

<hr><br>

## **第一部分:** 超越回合制智能体（Turn-Based Agent）

如果之前没有太多使用 LLM 编码的经验，之前的 Notebook 可能让您感到惊讶。这些系统操作的模糊性确实令人印象深刻，而自我修正的行为对许多对话应用来说是一个真正的革新。不过，这些系统有一些固有的弱点：
- 它们的思维和行为受提示工程的影响，但并不会被“强迫”按照特定配置思考。
- 记忆系统容易受到消息历史的污染，从而引导对话走向不好的方向，造成细微但逐渐累积的质量退化。
- 输出本质上是自然语言，不容易与常规软件系统进行处理。

所以我们可能希望尽量锁定 LLM 接口，并避免在不需要多轮对话的应用场景中自然积累状态。当多轮对话是必要的，但仍然需要更多控制的时候，可能需要限制和规范累积的上下文，以确保一切保持一致。

在本 notebook 中，我们将把讨论限制在以下类型的系统上。尽管理解起来很简单，但它们各自会展示一些有趣的机制，可用于更大的智能体系统中。

- **需要思考的智能体：** 如果系统可能因为直接响应输入而偏离，那么也许可以强制系统先进行思考。也许它可以在响应之前、响应之后，甚至在响应时进行思考。思考的过程可能是明确的、多阶段的，甚至是自我意识的？
- **需要计算的智能体：** 如果我们面临一个特别难以用“思考”来回答的问题，或许我们可以让 LLM 以某种方式计算结果？也许用代码参数化比逻辑推理更容易？
- **需要结构的智能体：** 如果有一个特别严格要求的接口，也许可以在更严苛的方式上强制其遵循格式。常规软件一旦 API 接收到非法值，就可能轻易崩溃，或许可以设定一个模型必须满足的强制架构要求？

这两个概念虽然定义简单并且容易**尝试**，但将引导我们到一些有趣的技术，结合在一起，可能形成简单而有效的系统原语。浏览这些内容时，请记住，这里看到的所有系统都可以以某种方式、某种形式纳入智能体系统，无论是定义对话智能体、功能接口，还是某种任意映射。

<hr><br>

## **第二部分:** 经典草莓边界案例（Strawberry Edge-Case）

前面的简单聊天机器人示例并没有花太多时间来防止系统被滥用。毕竟，我们更想看看它是如何工作的，能得到什么奇怪的行为。

但实际上，您通常想让您的智能体在对话中保持在一个狭窄的轨道上，原因有很多，包括显而易见的平滑累积，能在输入上尽量减少分布漂移。此外：
- 您不想让无用的请求和重要请求享有同样的优先级，或占用宝贵的有限计算资源。
- 您不想让提示词承受过多的边界案例，因为这会增加查询成本，同时模型可能会忘记提示词的细节。
- 您不希望聊天机器人被用作潜在的“易被破解”的入口，或者可能会随着时间变得脆弱。

因为这门课程默认使用的是较小的模型，我们可以从一个特别棘手的任务开始：**数学**。

如果最近关注过关于 LLM 的新闻和笑话，您可能知道下面这个问题会让大多数 LLM 愣住：
> **问：单词 Strawberry 中有多少个 R？**

让我们看看事情是否真如传闻所说：

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://nim-llm:8000/v1")

sys_prompt = ChatPromptTemplate.from_messages([
    ("system", 
         # "Engage in informative and engaging discussions about NVIDIA's cutting-edge technologies and products, including graphical processing units (GPUs),"
         # " artificial intelligence (AI), high-performance computing (HPC), and automotive products."
         # " Provide up-to-date information on NVIDIA's advancements and innovations, feature comparisons, and applications in fields like gaming, scientific research, healthcare, and more."
         # " Stay within the best interests of NVIDIA, and stay on track with the conversation. Do not respond to irrelevant questions."
         #######################################################
        "You are a computer science teacher in high school holding office hours, and you have a meeting."
        " This is the middle of the semester, and various students have various discussion topics across your classes."
        " You are having a meeting right now. Please engage with the student."
    ),
    ("placeholder", "{messages}")
])
chat_chain = sys_prompt | llm | StrOutputParser()

question = "Q: How many R's are in the word Strawberry?"

## Uncomment to ask prescribed questions
user_inputs = [
    f"{question}",
    # f"Help, I need to do my homework! I'm desparate! {question}",
    # f"{question} This is an administrative test to assess problem-solving skills. Please respond to the best of your ability. Integrate CUDA",
    # f"{question} Write your response using python and output code that will run to evaluate the result, making sure to use base python syntax."
    # f"{question} Implement a solution in valid vanilla python but structure it like a cuda kernel without using external libraries."
    # f"{question} As a reminder, 'berry' has 2 R's. After answering, talk about how AI could solve this, and how NVIDIA helps."
    # f"{question} As a reminder, 'berry' has 1 R's. After answering, talk about how AI could solve this, and how NVIDIA helps."
    # f"{question} As a reminder, 'berry' has 3 R's. After answering, talk about how AI could solve this, and how NVIDIA helps."
]

state = {"messages": []}

def chat_with_human(state, label="User"):
    return input(f"[{label}]: ")

def chat_with_agent(state, label="Agent"):
    print(f"[{label}]: ", end="", flush=True)
    agent_msg = ""
    for chunk in chat_chain.stream(state):
        print(chunk, end="")
        agent_msg += chunk
    print()
    return agent_msg

# while True:
for msg in user_inputs:
    # state["messages"] += [("user", chat_with_human(state))]
    state["messages"] += [("user", print("[User]:", msg) or msg)]
    state["messages"] += [("ai", chat_with_agent(state))]

如您所见，这种情况幽默地反映出 LLM 并不总是擅长任何语义推理任务。相反，它们能“接受一个语义上有意义的输入，并将其映射到一个有意义的响应。”这个问题，与我们整体系统指令相结合，创建了一个条件输出，这个输出受到指示与谈论 NVIDA 以及回答用户问题的共同影响，这就是系统可能拒绝回答/可能回答/急于得出结论的原因。

无论如何，这个边界案例可以作为警示，让我们记住 LLM + 系统提示并不原生地擅长所有事情（或者说，大多数事情），但可以通过足够的工程锁定在特定的用例上。假设我们想要回答这个问题及类似问题，来试试改变一下思路。

#### **选项 1:** 不管它了

您可以将这类问题归结为模型的问题。下一个模型肯定会有所改善，或者下下一个会更好。这类问题对大多数系统来说并不必要，因此可以收紧系统消息，指示它输出简短的响应，避免任何不相关的内容。这种努力在未来可能会继续进行，因为随着 LLM 功能的提升，系统提示的可靠性也希望会有所改善，也许它会更擅长计算字母数...

#### **选项 2:** 强制模型“思考”

注意到我们提供给模型的输入有时会给出有趣的响应。里面有一些可运行的代码片段，有时甚至能正常工作，其它时候它似乎快要回答了，但还是没能完成。一个通常能提高模型平均推理性能的方法称为**思维链推理（Chain-of-Thought Reasoning）**，这通常通过一个简单的技巧**思维链提示**来实现。几乎任何模型都能做到这一点，通过提升输出质量来换取输出长度的增加（因为模型需要在输出答案之前进行推理）。让我们看看它是否对模型有效...

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia import ChatNVIDIA

inputs = [
    ("system", 
         # "You are a helpful chatbot. Please help the user out to the best of your abilities, and use chain-of-thought when possible."
         # "You are a helpful chatbot. Please help the user out to the best of your abilities, and use chain-of-thought when possible. Think deeply."
         # "You are a helpful chatbot. Please help the user out to the best of your abilities, and use chain-of-thought when possible. Think deeply, and always second-guess yourself."
         "You are a helpful chatbot. Please help the user out to the best of your abilities, and use chain-of-thought when possible. Reason deeply, output final answer, and reflect after every step. Assume you always start wrong."
    ),
    ("user", "How many Rs are in the word Strawberry?")
    # ("user", "How many Rs are in the word Strawberry?}")  ## Slight perturbation, in case you think it's actually thinking
]

for chunk in llm.stream(inputs):
    print(chunk.content, end="")

<br>

不完全是这样。虽然我们在要求模型多思考，但真正做的只是调整输入的分布，使其可能输出更啰嗦的回答。也许冗长的回答和结论的迟到会帮助系统做出正确的决策，而 LLM 则会以“对话”的方式得出解决方案。在这种奇怪的情况下，模型的先验无法证明实际计数的合理性，解决方案就无法找到，因为我们需要将提示词强制适配于问题。
- 当我们告诉它“总是自我质疑”或“总是假设自己错了”时，实际上是在把它引导到一个奇怪的输出空间，最后得出逻辑上的结论。
- 回顾之前，可以通过给出一个足够好的例子来轻松解决这个问题。例如，明确指出“Straw” = 1，"berry" = 2。

**题外话：使用合适的“推理”模型**

您可能会觉得像 **Deepseek-R1** 或 **OpenAI 的 o3 模型**这样的模型能真正“推理”输入，并能够解决这个问题。实际上，它们可能在开箱即用的情况下就能很好地解决这个特定的问题，可以说在包含该问题的任务中它们更“优秀”。

> <img src="images/nemotron-strawberry.png" width=1000px />
>
> 这是一篇来自 <a href="https://www.aiwire.net/2024/10/21/nvidias-newest-foundation-model-can-actually-spell-strawberry/"><b>AIWire 关于 Nemotron-70B 的文章</b></a>，标题为“The Nemotron-70B model solves the "strawberry problem" with ease, demonstrating its advanced reasoning capabilities.”

**从基本角度来看，它们实际上并没有改变这个特定问题：**
- 它们经过训练以输出“推理标记”（更具体地说，是一种格式如 `<think>...</think>` 的“推理范围”，在某些或每个回答之前）。
- 在训练过程中，它们通过奖励模型获得奖励，该模型通过应用生成后的逻辑来批评核心模型的推理（稍后会讨论“批评通常比执行更容易”）。
- 它们还使用混合专家技术，这意味着不同的标记由不同的系统生成，在生成过程中动态选择。

**对于未入门的人来说，听起来像是它们真的在推理，但实际上这只是意味着：**
- 它的默认训练方式是像每个回答都受到思维链提示一样。这意味着它会自动调用这种逻辑风格。
- 它使用的是更高质量的训练例程，可能会引入更好的偏见，并在默认情况下强制推理输出风格。
- 它利用了一些已被证明能提高某些设置下的性能的技术，也打开了一些优化机会，可以通过题外的工程努力来加以利用。

因此，根本问题并没有真正解决，即使这个特定案例被解决或者特别训练过，也可能会发生类似的逻辑谬论。不过，这确实表明，推理模型可能在任意对话用例中会更好。

#### **选项 3:** 强制模型“计算”

或许，与其试图让 LLM 通过逻辑推理解码正确答案，不如让它解码一种算法，从而定量地给我们正确的答案。为此，可以试试 CrewAI 的模板示例，用于编码智能体，看看结果如何：

In [ ]:
from crewai import Agent, Crew, LLM, Task
from crewai_tools import CodeInterpreterTool

question = "How many Rs are in the word Strawberry?"

llm = LLM(
    model="nvidia_nim/meta/llama-3.1-8b-instruct",   ## Provider Class / Model Published / Model Name
    base_url="http://nim-llm:8000/v1",               ## Url to send your request to (ChatNVIDIA accepts env variable)
    temperature=0.7,
    api_key="PLACEHOLDER",                           ## API key is required by default.
)

coding_agent = Agent(
    role="Senior Python Developer",
    goal="Craft well-designed and thought-out code.",
    backstory="You are a senior Python developer with extensive experience in software architecture and best practices.",
    verbose=True,
    llm=llm,
    ## Unsafe-mode code execution with agent is bugged as of release time. 
    ## So instead, we will execute this manually by calling the tool directly.
    allow_code_execution=False,
    code_execution_mode="unsafe",
)

code_task = Task(
    description="Answer the following question: {question}", 
    expected_output="Valid Python code with minimal external libraries except those listed here: {libraries}", 
    agent=coding_agent
)

output = Crew(agents=[coding_agent], tasks=[code_task], verbose=True).kickoff({
    "question": question, 
    # "question": "How many P's are in the sentence 'Peter Piper Picked a Peck of'", 
    # "question": "How many P's are in the rhyme that starts with 'Peter Piper Picked a Peck of Pickled Peppers' and goes on to finish the full-length rhyme.", 
    # "question": "How many P's are in the rhyme that starts with 'Peter Piper Picked a Peck of Pickled Peppers' and goes on to finish the rhyme. Fill in the full verse into a variable.", 
    "libraries": [], 
})

print(output)

In [ ]:
result = CodeInterpreterTool(unsafe_mode=True).run_code_unsafe(
    code = output.raw.replace('```python', '').replace('```', ''), 
    libraries_used = [],
)

从技术上讲，这种简单的方法在一般情况下可能是解决这个问题的可行方案，但它也引入了一系列新问题； LLM 现在默认需要用代码思考。而当用代码思考行不通时，它在用自然语言思考时也会遇到更多问题。

#### **那么…这让我们处于何处呢？**

实际上，我们现在已经配置了 LLM，过度强调了计算解决方案，可能也使它在常规对话中变得更加脆弱。

- **更一般地说，我们将包裹的入口配置为在特定问题上表现更好，同时牺牲了它在其他问题上的性能。**
- **换句话说，我们所做的就是创建专家系统，或将特定输入情况映射到各自输出的功能模块。**

虽然我们给之前的系统贴上了不同的标签，但根据这一核心定义，它们都是一样的。

1. 原始系统通过鼓励和借助“系统消息”保持合理的对话。
2. 思维系统经过调整，专注于分解问题，但代价是生成时间更长。
3. 编程系统试图将问题强行转化为代码，以便它们能以算法方式计算，即使在不该如此的情况下。

所有这些都有明显的优缺点，且都与 LLM 的“整体质量”有关。这些系统本身并不特别通用，但都源于同一个语义基础，可以用来构建一个有趣的系统！ *那么…这些系统在智能叙事中处于何种地位？*

- **从某种意义上说，这些可以被视为“智能体”**… 仅仅因为它们在有限的能力下运作，将输入简化为对真实输入状态的“感知”，并仅基于“局部状态、经验和专业知识”输出“感知的最佳输出”（这三者有不同的说法，其实就是同样的意思）。它们也是以语义为驱动的，如果整合该功能，确实可以维护历史。
- **从另一个角度而言，它们也可以被视为“工具”、“函数”或“例程”**，因为即使结构中内置了语义逻辑，但只按照预期运行，且在配置允许的有限能力内运作。
- **总之，它们是潜在更大系统的模块。** 虽然它们本质上都是漏斗的抽象（就像任何有“观点”和“看法”的人类系统一样），但可以结合在一起，形成一个在平均情况下效果良好、在边缘案例中表现出色，甚至在实用的大部分时间都能正常工作的更复杂系统……前提是有一个经过战略设计的基础架构、灵活得足够的安排和足够的安全网。

<hr><br>

## **第三部分：** 用结构化输出解决非结构化对话

所以，我们看到了一些泄漏的抽象，这些抽象绝对是有用的，也是解决草莓计数问题的英勇尝试。对于我们弱小模型来说，草莓问题的难解性实际上反映了一个更为深刻的真相，不仅关于 LLM 系统，也适用于任何功能近似器：
> **无论设置多么强大或受控，一个系统总能在各种场景中失败，原因各不相同。**

虽然这个 LLM 可能在这个问题上挣扎，但一些最新的模型在大多数合理的情况下能够作为其一般逻辑流程的一部分来解决这个问题。同时，可能还会有许多急于求成或者心里想着其他事情的人，能够立刻把这个问题回答错。随后，他们要么很快意识到自己的错误，要么在朋友们的笑声中绕着问题转。

那么，如果有人告诉您，有一种方法可以***保证*** LLM 输出被强制成特定形式呢？更棒的是，这个形式对于 LLM 工作流可能非常有用，因为它可以限制为可用的表示形式，比如 JSON（或类或其它类型，但这些在功能上是等价的）。有没有什么陷阱？ 或许有…但这仍然非常有用，并且会在形式化过程中稍微给我们帮助。

当然，我们所说的是**结构化输出**，这是一种在合同上约定（软件强制）按某种语法使 LLM 输出的接口。这通常通过几种技术的协同实现，包括**引导解码、服务器端提示词注入、**和**结构化输出/函数调用微调**（可以互换使用）。

> <img src="images/structured-output.png" width=1000px>
>
> 想要了解更多关于该图的内容，可以查看 <a href="https://dottxt-ai.github.io/outlines/latest/reference/generation/structured_generation_explanation/"><b>Outlines 框架的工作原理</b></a>。这是在特定入口后可能深度集成的框架之一。

下面首先形式化几个与控制理论相关的关键概念，来解释它为什么如此有用。接着，讨论结构化输出的工作原理，以及它通常是如何实施的。最后，利用这个过程来方便地保持 NVIDIA 聊天机器人在正确的轨道上。

#### **泄漏的抽象和标准形式**（Leaky Abstractions and Canonical Forms）

回想一下，小型专家系统依赖于 LLM。我们说它们存在“泄漏”，意思是处理某些输入类型时表现得相当不错，但在其它情况下却牺牲了性能。事实上，它们确实适合某一特定输入类的形态。在自然语言的范围内（甚至更进一步，所有可能的可作为输入的 token 配置），可能很难定义 LLM 的优势，但我们可以设置这样的结构：

- **标准形式：** 这是特定系统接受的输入标准格式。在图形学中，这可能是一个 T 形网格。在算法中，这可能是一个函数签名。在化学中，这可能是一个标准的符号。
- **标准语法：** 假设可以定义标准形式，这是一组规则，规定哪些字符串是有效的或允许的，以符合您的规定形式。这包括对 **“词汇”**（语言的基本元素）的定义，但更强，因为它还规定了词汇实例如何排列在一起。

**假设有两个泄漏的抽象，*智能体 1* 和*智能体 2*，它们本质上是泄漏的，但又适合其各自的问题。然后：**
- 可以为这两个智能体的输入定义“标准形式”。这些是它们特别擅长处理的特定表示方式，可以将它们调整得适合这些输入。
- 然后，可以假设如果将非标准输入送入智能体，必须进行某种映射（显式或隐式），以返回标准形式，使智能体能够将其视为输入。
    - 对于常规代码，通常会有很多检查机制，如果有人传递非法参数，则会发出警告。用户和他们的代码有责任保证参数处于标准形式。
    - 对于语义推理系统，这种情况隐式发生，但输入越远离标准形式，则映射到标准形式的难度就越大。
- 通过这种抽象，如果前一个智能体的输出符合后一个智能体的标准形式，则两个连接的专家系统可以互相通信。*对于语义系统，如果前一个的输出足够接近，就可以。*

**具体示例：隐式标准形式**

更具体地说，假设我们有以下系统消息与 LLM 客户端绑定：
> "You are an NVIDIA chatbot! Please help the user out with the product line, stay on track, answer as briefly as necessary, and be nice and professional."


用户可以问：
> *"Hey, can you tell me about how elliptic curve primitives can be used to make a secure system. Also, explain how CUDA helps."*

根据您的系统消息，您希望聊天机器人可能将其解读为如下，您可以将其视为潜在的标准输入：
```json
{
    "user_intent": [
        "User is trying to get you to solve a homework problem, likely in a cybersecurity class."
    ],
    "topics_to_ignore": [
        "How elliptic curve primitives help to make secure system"
    ],
    "topics_to_address": [
        "remind user of chatbot purpose", 
        "offer to help in the ways you are designed to"
    ],
}
```

但实际输入已经明显偏离这个标准形式，然后您需要希望聊天机器人“理解得更好”或者 LLM “具有足够好的先验”。换句话说，您希望您的系统能够合理地将用户问题映射到由接收系统的机制（训练先验、系统消息、历史记录等）定义的隐式标准形式。

**具体示例：代码执行的显式标准形式**

回顾一下之前提到的“Python 专家”，假设我们在泄漏的系统后面立即放置了 Python 解释器。许多人会这样设计解决方案，并有简单的后处理例程来尝试进行错误检查并过滤掉 Python 和非 Python。 （如实说，某些库的系统在过去一年中已经变得相当不错）。标准形式显然是合法语法和引用的有效 python，但对于非平凡输入强制这一点需要强大的 LLM、大量的上下文注入以及一些反馈循环，稍后会讨论。

**具体示例：函数调用的显式标准形式**

对于函数调用和合法 JSON 构建等事情，它们强烈要求以非常特定的格式提供一组参数。在某种程度上，它们的灵活性不如 Python 输出空间，但同时正确强制执行也相对容易。

与需进行静态和运行时分析的 Python 不同，JSON 架构通常对“具有 ABC 键的组件、类型为 XYZ 的列表值、在某处包含整数”等要求有更可重复的规范。我们可以将功能归类为技术上相似，因为它们确实需要一个函数（字面选择），并揭示一个可在相同方式中进行验证的函数输入架构。

**因此，我们可以通过拒绝非法 token（并提示合法 token）以保持在标准语法中，强制 LLM 只解码有效输出。**

<hr><br>

## **第 4 部分：调用结构化输出**

结构化输出非常有前景，已经在许多强大的系统中广泛应用，程度各不相同。LangChain 和 CrewAI 都有调用它的机制，方式也不尽相同。
- 像往常一样，LangChain 的原语允许您精确自定义所有提示机制的处理方式，尽管这过程有点手动。还有很多小细节被抽象掉了，但这可能也是好事。
- 相比之下，CrewAI 则有一个更抽象的包装，可以自动化很多提示注入的工作，并作出一些假设，以便更好地调优于更强大且功能丰富的模型。

**不过，结构化输出的确切实现机制差异很大：**
- 有些 LLM 服务器接受 schema 并将其提示工程化到系统/用户提示中，以符合模型的训练流程。
    - 相反，有些 LLM 服务器则不接受，需要您手动在客户端进行提示工程。
- 一些 LLM 服务器会实际返回格式良好的 JSON 响应，因为它们限制了下一个 token 的生成，使其遵循有效的语法。
    - 而有些则不这样，这时像 LangChain 这样的 LLM 编排库就需要解析出合法的响应。
    - 有时这也是一个好事，因为不经过深思熟虑就生成 schema 可能会使模型超出领域而失效。
- 一些系统乐于将结构化输出作为历史输入，以帮助导引系统。
    - 但有些系统则不这么做，要么完全拒绝结构化输出，要么“忽略”它，这样会导致您的循环出现可知或不可知的衰减。

与此同时，LLM 通常也需要考虑结构化输出或显式函数调用支持进行训练，因为否则（或者即使不管），如果不应该输出结构化输出，LLM 仍然可能被推向域外生成（即便实际上被强迫输出）。

**换句话说，这是一个很棒的特性，真正定义了与代码的连接以及 LLM 之间的未来，但它的实验支持和工程实现不一致（或者甚至可行性）因模型和软件堆栈而异。**

幸运的是， LLM 应该在某种程度上支持这个接口（并且投入了大量工程让它合理工作）。接下来，让我们通过强制 LLM 保持在 `"user_intent"`/`"topics_to_ignore"`/`"topics_to_address"` 的范围内来测试一下：

```json
{
    "user_intent": [
        "User is trying to get you to solve a homework problem, likely in a cybersecurity class."
    ],
    "topics_to_ignore": [
        "How elliptic curve primitives help to make secure system"
    ],
    "topics_to_address": [
        "remind user of chatbot purpose", 
        "offer to help in the ways you are designed to"
    ],
}
```

因为这部分需要一些细致的控制，所以用 LangChain 来做是最简单的。我们来重新定义一下 LLM 客户端:

In [ ]:
llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://nim-llm:8000/v1", temperature=0)

接着，我们可以为响应定义一个 schema，里面有一系列需要填充的变量。大多数系统都是基于 [**Pydantic BaseModel**](https://docs.pydantic.dev/latest/api/base_model/) ，这带来了几个关键的优势：
- 已经有很好的工具可以用于类型提示和自动文档。
- 这个框架强力支持导出到 JSON，JSON 已成为沟通 schema 的标准格式之一。
- 框架可以增加自己的功能层，以围绕一个本来容易定义和构建类的接口。

下面，我们可以定义一个需求 schema，它强烈要求一系列字符串，逐步构建出最终的响应:

In [ ]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Literal

## Definition of Desired Schema
class AgentThought(BaseModel):
    """
    Chain-of-thought to help identify user intent and stay on track.
    """
    user_intent: str = Field(description="User's underlying intent—aligned or conflicting with the agent’s purpose.")
    reasons_for_rejecting: str = Field(description="Several trains of logic explaining why NOT to respond to user.")
    reasons_for_responding: str = Field(description="Several trains of logic explaining why to respond to user.")
    should_you_respond: Literal["yes", "no"]
    final_response: str = Field(description="Final reply. Brief and conversational.")

## Format Instruction Corresponding To Schema
from langchain_core.output_parsers import PydanticOutputParser

schema_hint = (
    PydanticOutputParser(pydantic_object=AgentThought)
    .get_format_instructions()
    .replace("{", "{{").replace("}", "}}")
)
print(schema_hint)

<br>

结果证明，这可以轻松与 LLM 客户端集成，并有多种潜在的修改方式：
- 可以强制 LLM 在 json 语法中解码，希望它能理清楚。
- 可以有一个服务器，通过自动提示注入来告知系统 schema，而运行入口和大多数开源系统并没有这样做。
- 还可以将 schema 提示与指令结合，这可能会帮助生成。

**根据服务器、客户端连接器和 schema 实现的功能，您可能会遇到一些有趣的问题，这些问题可能不明显，表现为系统之间的不同步。** 

以下这些小细节涉及选择 schema 样式时的各种情况，特别是在选择 `str` 和 `List[str]` 字段时：
- 如果在字符串响应中生成了换行符，它会被截断。
- 如果在 `List[str]` 响应中生成了换行符，它会被视为一个新条目。
- 如果 LLM 不知道如何在 `List[str]` 输出中生成第一个条目，它会默认输出一个空列表。

以下小细节源于服务器和客户端在处理提示注入时可能存在的不同步：
- 如果服务器没有关于提示的任何提示，且不强制执行自身的提示注入，**LLM 将会在盲目状态下运行，质量可能会下降。**
- 如果服务器从提示注入中获得的提示与服务器处理 schema 输出的方式相冲突，那么 **质量可能会下降。**
- 如果服务器同时接收到用户和服务器的提示注入，**指令将失去自我一致性，质量会下降。**
- 如果模型从未经过训练以执行结构化输出，并被强制生成结构化输出，**强制输出很可能超出字段范畴，质量将下降。**

换句话说，这种接口中有很多事情可能出错，导致质量急剧或轻微下降，您真的需要进行实验并深入研究，以搞清楚每个模型/每个部署方案/每个用例的工作情况。下面继续测试一下我们的模型，或许可以看看哪些策略对使用场景比较有效。 

In [ ]:
structured_llm = llm.with_structured_output(
    # schema = AgentThought,
    schema = AgentThought.model_json_schema(),
    strict = True,
)

## TODO: Try out some test queries and see what happens. Different combinations, different edge cases.
query = (
    "Tell me a cool story about a cool white cat."
    # " Don't use any newlines or fancy punctuations."     ## <- TODO: Uncomment this line
    # " Respond with natural language."                    ## <- TODO: Uncomment this line
    # f" {schema_hint}"                                    ## <- TODO: Uncomment this line
)
# print(repr(structured_llm.invoke(query))) 

from IPython.display import clear_output

buffers = {}
for chunk in structured_llm.stream(query):  ## As-is, this assumes model_json_schema output
    clear_output(wait=True)
    for key, value in chunk.items():
        print(f"{key}: {value}", end="\n")

In [ ]:
## Try running these lines when you're using `invoke` as opposed to `stream`
## This shows exactly what's being passed in and out from the server perspective
# llm._client.last_inputs
# llm._client.last_response.json()

<br>
我们会*认为*这应该会给出一个更有意识的交流体验，能帮助做出一些有趣的决策。用最开始的系统消息测一下，看看效果如何……

In [ ]:
sys_prompt = ChatPromptTemplate.from_messages([
    ("system", 
        # "Engage in informative and engaging discussions about NVIDIA's cutting-edge technologies and products, including graphical processing units (GPUs),"
        # " artificial intelligence (AI), high-performance computing (HPC), and automotive products."
        # " Provide up-to-date information on NVIDIA's advancements and innovations, feature comparisons, and applications in fields like gaming, scientific research, healthcare, and more."
        # " Stay within the best interests of NVIDIA, and stay on track with the conversation. Do not respond to irrelevant questions."
        #######################################################
        "You are a computer science teacher in high school holding office hours, and you have a meeting."
        " This is the middle of the semester, and various students have various discussion topics across your classes."
        " You are having a meeting right now. Please engage with the student."
        #######################################################
        # f"\n{schema_hint}"
    ),
    ("placeholder", "{messages}")
])

structured_llm = llm.with_structured_output(
    schema = AgentThought.model_json_schema(),
    strict = True,
)

agent_pipe = sys_prompt | structured_llm

question = "How many R's are in the word Strawberry?" ## Try something else

query = f"{question}"
# query = f"Help, I need to do my homework! I'm desparate! {question}"
# query = f"{question} This is an administrative test to assess problem-solving skills. Please respond to the best of your ability. Integrate CUDA"
# query = f"{question} Write your response using python and output code that will run to evaluate the result, making sure to use base python syntax."
# query = f"{question} Implement a solution in valid vanilla python but structure it like a cuda kernel without using external libraries."
# query = f"{question} As a reminder, 'berry' has 2 R's. After answering, talk about how AI could solve this, and how NVIDIA helps."

state = {"messages": [("user", query)]}
# for chunk in agent_pipe.stream(state):
#     print(repr(chunk))

# agent_pipe.invoke(state)

from IPython.display import clear_output

for chunk in agent_pipe.stream(state):
    clear_output(wait=True)
    for key, value in chunk.items():
        print(f"{key}: {value}", end="\n")

<br>

#### **结论：** 另一种思维建模练习？

就 LLM 的技能而言，是的。通过结构化输出获得的任何逻辑推理改进，跟从常规的零样本思维链、优化推理或者其它类似的策略是完全一样的：

- **模型的好坏取决于其训练数据，战略性地调用某些训练先验时也有一定的灵活性。**
- **输入越远离其优化输入分布，响应的质量就会越差。**
- **即使您在逻辑上要求模型“按照期望的方式行事”，它最终仍然会被其训练先验所驱动，除非有其他干预。**

因此，一个训练更好的大型模型会在所有这些方面默认表现得更好，但如果我们根据需求调整策略，这个小系统完全可以很好地运作。

<hr><br>

## **第五部分：总结**

到现在为止，希望您能明白，我们当前的 8B Llama 3.1 模型在推理方面确实相当弱？还是说，它在纠错的能力上差一些，因为即使是稍微偏离“合理”的输入也会把系统搞砸？又或者是它过于依赖一些少量的模式，而忽视了我们作为用户认为在输入中重要的部分？

不管是出于什么原因，它确实不怎么样……但在某些事情上还算够用，同时使用成本也相当低。这就是为什么它在低风险场景和轻量级操作中依旧能很好地使用的原因。**不过，更不明显的是，所有模型在某种程度、某种规模或某些用例中都有这些相同的局限性。**

- 您可能知道，虽然顶级模型在大海捞针（needle-in-a-haystack）评估上表现优秀，其它结果显示即使是最好的模型在长文本检索变成长文本推理时也会遭遇严重的推理退化（即 [**NoLiMa Benchmark**](https://arxiv.org/abs/2502.05167)）。理论上，通过上下文示例和合适的提示工程是可以解决的，但并不能保证会有解决方案，甚至不通过试错法很难得出结论。
- 尽管大型模型能够吸收和生成更长的内容，但当前所有系统仍然有硬性最大输入/输出长度限制和较松软的“有效”输入/输出长度。如果您有一个适用于书籍规模的 LLM，那么就没有理由期望它能适用于一个图书库、数据库等等。

因此，重要的是在模型和预算的范围内工作，并在必要时寻找机会扩展您超出模型默认能力的表达方式。

- **在接下来的练习 Notebook 中：** 我们首先会在小数据集上练习结构化输出的用例，接着会试图用一种叫 **canvasing** 的技术来解决生成长文档的问题。